In [1]:
import torch

$$
\text{Inputs}: \mathbf{x} =
\begin{bmatrix}x^0 & x^1 & x^2 & \cdots & x^T\end{bmatrix}
$$

In [32]:
inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your     (x^0)
     [0.55, 0.87, 0.66], # journey  (x^1)
     [0.57, 0.85, 0.64], # starts   (x^2)
     [0.22, 0.58, 0.33], # with     (x^3)
     [0.77, 0.25, 0.10], # one      (x^4)
     [0.05, 0.80, 0.55]] # step     (x^5)
)

1. Compute the **attention scores**
$$\mathbf{w} = \begin{bmatrix}w_0 & w_1 & w_2 & \cdots & w_T\end{bmatrix}$$
where $w_i = \mathbf{x} \times x^i$

3. Compute the **context vector**
$$ 
\mathbf{z} = \begin{bmatrix}z_0 & z_1 & z_2 & \cdots & z_T\end{bmatrix}
$$
where $z_i = w_i \times \mathbf{x}$

---

**Example :**<br>
We use a query $q$, for the second input query $q = x^2 = [0.55, 0.87, 0.66]$ corresponding to the word *journey* :<br>
1. $w_2 = \mathbf{x} \times q = [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865]$
2. $z_2 = w_2 \times \mathbf{x} = [0.4419, 0.6515, 0.5683]$

In [14]:
# 1. choose an input query (q = x^k)
query = inputs[1]
# 2. compute the attention scores : dot product (w^i = x^i * q)
attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query)

# 3. normalize to get the attention weights
print(attn_scores_2)
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()

print(attn_weights_2_tmp, "sum=", attn_weights_2_tmp.sum())

# (3bis. we could use softmax normalization instead)
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
print(attn_weights_2, "sum=", attn_weights_2.sum())

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])
tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656]) sum= tensor(1.0000)
tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581]) sum= tensor(1.)


In [16]:
# 4. compute context vector (z^k = Σw^i * x^i)
context_vec_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i] * x_i

print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


---

In [36]:
# attention scores (w)
attn_scores = torch.empty(6, 6)
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)

attn_scores

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

In [39]:
attn_scores = inputs @ inputs.T
attn_scores

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

In [41]:
# normalization to get attention weights
attn_weights = torch.softmax(attn_scores, dim=-1)
attn_weights

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])

In [43]:
# context vectors (z)
all_context_vecs = attn_weights @ inputs
all_context_vecs

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

---

### Adding trainable weights